## Computer Vision Driving Car

### installing libs

In [ ]:
!pip install ultralytics opencv-python

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110543 sha256=dfcb40504bac0f03c1b9d70dfdf53e7b7f70db1e000fcfc725ac305cc536d320
  Stored in directory: /home/kakashi/.cache/pip/wheels/77/bf/4c/b0c3f4798a0166668752312a67118b27a3cd341e13ac0ae6ee
Successfully built filterpy

[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip


## Config

In [66]:
ALLOWED_CLASSES = {
    0: "Pedestrian",
    2: "Car",
    3: "Motorcycle",
    5: "Bus",
    7: "Truck"
}
CLASS_COLORS = {
    0: (255, 255, 0),    # Pedestrian - Cyan
    2: (0, 255, 0),      # Car - Green
    3: (255, 0, 255),    # Motorcycle - Magenta
    5: (0, 0, 255),      # Bus - Red
    7: (255, 165, 0)     # Truck - Orange
}
MODEL_NAME = "yolov8n.pt"
VIDEO_PATH = "challenge-mle2.mp4"
OUTPUT_VIDEO = "output_tracking.mp4"
OUTPUT_CSV = "tracking_results.csv"

## IMPORTS

In [57]:
import cv2
import time
from ultralytics import YOLO
import numpy as np
from pathlib import Path


## YOLOv8 + ByteTrack Detection & Tracking Pipeline

In [61]:
import cv2
import time
import csv
from ultralytics import YOLO


def tracking_1(
    video_path: str,
    output_video_path: str,
    output_csv_path: str,
    model_name: str = "yolov8n.pt",
    conf: float = 0.4,
    iou: float = 0.5
):
    model = YOLO(model_name)

    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), "Cannot open input video or cant find the video"

    # Lock frame size using first frame (CRITICAL)
    ret, frame = cap.read()
    assert ret, "Failed to read video frames"

    height, width = frame.shape[:2]

    # Safe FPS
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 1 or fps > 120:
        fps = 30

    # MP4 writer
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    assert out.isOpened(), "error in saving video"

    csv_file = open(output_csv_path, mode="w", newline="")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(
        ["frame_id", "object_id", "class", "x1", "y1", "x2", "y2"]
    )

    prev_time = time.time()
    frame_id = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (width, height))

        results = model.track(
            frame,
            persist=True,
            tracker="bytetrack.yaml",
            conf=conf,
            iou=iou,
            verbose=False
        )

        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu().numpy()
            ids = results[0].boxes.id.cpu().numpy()
            classes = results[0].boxes.cls.cpu().numpy()

            for box, track_id, cls_id in zip(boxes, ids, classes):
                cls_id = int(cls_id)
                if cls_id not in ALLOWED_CLASSES:
                    continue

                x1, y1, x2, y2 = map(int, box)
                object_id = int(track_id)
                class_name = ALLOWED_CLASSES[cls_id]

                # write csv row
                csv_writer.writerow([
                    frame_id,
                    object_id,
                    class_name,
                    x1, y1, x2, y2
                ])

                color = CLASS_COLORS[cls_id]
                label = f"{class_name} | ID {object_id}"

                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(
                    frame,
                    label,
                    (x1, y1 - 8),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    color,
                    2
                )

        # FPS overlay
        now = time.time()
        fps_text = f"FPS: {1 / (now - prev_time + 1e-6):.1f}"
        prev_time = now

        cv2.putText(
            frame,
            fps_text,
            (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            2
        )

        out.write(frame)
        cv2.imshow("YOLOv8 + ByteTrack", frame)

        frame_id += 1

        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    out.release()
    csv_file.close()
    cv2.destroyAllWindows()

    print(f"Video saved to: {output_video_path}")
    print(f"CSV saved to: {output_csv_path}")


In [62]:
tracking_1(
    video_path=VIDEO_PATH,
    output_video_path=OUTPUT_VIDEO,
    output_csv_path=OUTPUT_CSV,
    model_name=MODEL_NAME
)


Video saved to: output_tracking.mp4
CSV saved to: tracking_results.csv


In [63]:
from IPython.display import Video

Video('output_tracking.mp4', width=640, height=360)

In [64]:
import cv2
import time
import csv
from ultralytics import YOLO


def tracking_2(
    video_path: str,
    output_video_path: str,
    output_csv_path: str,
    model_name: str = "yolov8n.pt",
    conf: float = 0.4,
    iou: float = 0.5
):

    model = YOLO(model_name)

    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), "Cannot open input video or cant find the video"

    # Lock frame size
    ret, frame = cap.read()
    assert ret, "Failed to read frames"
    height, width = frame.shape[:2]

    # Safe FPS
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 1 or fps > 120:
        fps = 30

    # MP4 writer
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    assert out.isOpened(), "can not save the video"

    # CSV setup
    csv_file = open(output_csv_path, mode="w", newline="")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["frame_id", "object_id", "class", "x1", "y1", "x2", "y2"])

    prev_time = time.time()
    frame_id = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (width, height))

        results = model.track(
            frame,
            persist=True,
            tracker="botsort.yaml",   # BoT-SORT with ReID
            conf=conf,
            iou=iou,
            verbose=False
        )

        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu().numpy()
            ids = results[0].boxes.id.cpu().numpy()
            classes = results[0].boxes.cls.cpu().numpy()

            for box, track_id, cls_id in zip(boxes, ids, classes):
                cls_id = int(cls_id)
                if cls_id not in ALLOWED_CLASSES:
                    continue

                x1, y1, x2, y2 = map(int, box)
                object_id = int(track_id)
                class_name = ALLOWED_CLASSES[cls_id]

                # Write CSV row
                csv_writer.writerow([
                    frame_id,
                    object_id,
                    class_name,
                    x1, y1, x2, y2
                ])

                # Draw
                color = CLASS_COLORS[cls_id]
                label = f"{class_name} | ID {object_id}"

                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(
                    frame,
                    label,
                    (x1, y1 - 8),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    color,
                    2
                )

        # FPS overlay
        now = time.time()
        fps_text = f"FPS: {1 / (now - prev_time + 1e-6):.1f}"
        prev_time = now

        cv2.putText(
            frame,
            fps_text,
            (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            2
        )

        out.write(frame)
        cv2.imshow("YOLOv8 + BOT-SORT optimized algorithm", frame)

        frame_id += 1

        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    out.release()
    csv_file.close()
    cv2.destroyAllWindows()

    print(f"Video saved to: {output_video_path}")
    print(f"CSV saved to: {output_csv_path}")


In [65]:
tracking_2(
    video_path=VIDEO_PATH,
    output_video_path=OUTPUT_VIDEO,
    output_csv_path=OUTPUT_CSV,
    model_name=MODEL_NAME
)


Video saved to: output_tracking.mp4
CSV saved to: tracking_results.csv
